<a href="https://colab.research.google.com/github/aviax1/AE1/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**dependencies**

In [68]:
# used snniped from https://github.com/L1aoXingyu/pytorch-beginner/
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
from tensorflow.keras.datasets import mnist
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np

**initial**

In [74]:
(xtrain,ytrain), (xtest,ytest) = mnist.load_data()
num_epochs=200      #
batch_size = 20    #
image_size=784      #
hidden_size=128     #
lv_size = 64        # Latent Variable 
learning_rate=1e-4  #
cret = nn.MSELoss() # criterion

**build model**

In [58]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(image_size, image_size),nn.ReLU(True), nn.Linear(image_size, hidden_size),
            nn.ReLU(True), nn.Linear(hidden_size, hidden_size), nn.ReLU(True), nn.Linear(hidden_size, lv_size))
        self.decoder = nn.Sequential(
            nn.Linear(lv_size, hidden_size),nn.ReLU(True),nn.Linear(hidden_size, hidden_size),
            nn.ReLU(True),nn.Linear(hidden_size, hidden_size),nn.ReLU(True), nn.Linear(hidden_size, image_size), nn.Tanh())

    def forward(self, x):
        return self.decoder(self.encoder(x))

**model setting**

In [69]:
model = autoencoder()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

class DigitDataSet(Dataset):
  def __init__(self, dataset):
      self.dataset = dataset
      self.transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.5], [0.5])])

  def __len__(self):
      return len(self.dataset)

  def __getitem__(self, idx):
      if torch.is_tensor(idx):
          idx = idx.tolist()
      return self.transform( self.dataset[idx,:,:])
       

**train model by digit**

In [73]:
for by_digit in range(10):
  print("*****\nstart traning Model for digit " +str(by_digit) +"\n")
  dataloader = DataLoader(DigitDataSet(x[ytrain==by_digit]), batch_size=batch_size,shuffle=True, num_workers=4)
  
  for epoch in range(num_epochs):
    for data in dataloader:
      imgs = data
      imgs = Variable(imgs.view(imgs.size(0), -1))
      output = model(imgs)
      loss = criterion(output, imgs)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    print('epoch [{}/{}], loss:{:.4f}' .format(epoch + 1, num_epochs, loss.data))
    

  torch.save(model.state_dict(), './ae_'+by_digit+'.pth')
  print("\nfinish traning Model Number " +str(by_digit) +"\n")
  print("*****\n")

*****
start traning Model for digit 0

epoch [1/200], loss:0.2395
epoch [2/200], loss:0.2426
epoch [3/200], loss:0.2029
epoch [4/200], loss:0.1859
epoch [5/200], loss:0.1844
epoch [6/200], loss:0.1472
epoch [7/200], loss:0.1484
epoch [8/200], loss:0.1378
epoch [9/200], loss:0.1292
epoch [10/200], loss:0.1516
epoch [11/200], loss:0.1271
epoch [12/200], loss:0.1334
epoch [13/200], loss:0.1119
epoch [14/200], loss:0.1434
epoch [15/200], loss:0.1203
epoch [16/200], loss:0.1134
epoch [17/200], loss:0.1061
epoch [18/200], loss:0.1036
epoch [19/200], loss:0.1099


KeyboardInterrupt: ignored